In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
from utils import *

In [2]:
import requests
from bs4 import BeautifulSoup

url = 'https://2019.emnlp.org/program/schedule/'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

soup

<!DOCTYPE html>

<!--
  Minimal Mistakes Jekyll Theme 4.8.0 by Michael Rose
  Copyright 2017 Michael Rose - mademistakes.com | @mmistakes
  Free for personal and commercial use under the MIT license
  https://github.com/mmistakes/minimal-mistakes/blob/master/LICENSE.txt
-->
<html class="no-js" lang="en">
<head>
<meta charset="utf-8"/>
<!-- begin SEO -->
<title>Main Conference Schedule - EMNLP-IJCNLP 2019</title>
<meta content="EMNLP-IJCNLP 2019 Conference Schedule." name="description"/>
<meta content="Your Name" name="author"/>
<meta content="en" property="og:locale"/>
<meta content="EMNLP-IJCNLP 2019" property="og:site_name"/>
<meta content="Main Conference Schedule" property="og:title"/>
<link href="https://2019.emnlp.org/program/schedule/" rel="canonical"/>
<meta content="https://2019.emnlp.org/program/schedule/" property="og:url"/>
<meta content="EMNLP-IJCNLP 2019 Conference Schedule." property="og:description"/>
<meta content="@emnlp2019" name="twitter:site"/>
<meta content="Main 

In [3]:
import re

session_endings_to_remove = [' I', ' II', ' III', ' IV']
elements = soup.select("#main > article > div > section > p")

data = []
for element in elements:
    m = re.match('.*Session .*: (.*)', element.text)
    if m:
        session = m[1].strip()
        for ending in session_endings_to_remove:
            if session.endswith(ending):
                   session = session.removesuffix(ending)
        list_of_papers = element.find_next_sibling()
        
        if 'Poster & Demo' in element.text:
            list_of_papers = list_of_papers.find('ul')
            papers = list_of_papers.find_all('li')
            for paper in papers:
                paper_text = str(paper.text).strip()
                paper_match = re.match('(.*)\(# (.*)\)?', paper_text)
                title = paper_match[1].strip()
                paper_id = paper_match[2]
                data.append({ 'title': title, 'id': paper_id, 'area': session})
                    
        else:
            papers = list_of_papers.find_all('li')
            for paper in papers:
                paper_html = str(paper)
                paper_match = re.match('.*<br\/> (.*)\(# (.*)\).*', paper_html)
                title = paper_match[1].strip()
                paper_id = paper_match[2]
                data.append({ 'title': title, 'id': paper_id, 'area': session})
                

            
df = pd.DataFrame(data)
df

,title,id,area
0,Attending to Future Tokens for Bidirectional S...,1443,Machine Learning
1,Attention is not not Explanation,526,Machine Learning
2,Practical Obstacles to Deploying Active Learning,1176,Machine Learning
3,Transfer Learning Between Related Tasks Using ...,1207,Machine Learning
4,Insertion-based Decoding with automatically In...,TACL-1732,Machine Learning
...,...,...,...
731,Spelling-Aware Construction of Macaronic Texts...,3639),"Information Extraction, Text Mining and NLP Ap..."
732,Towards Machine Reading for Interventions from...,3718),"Information Extraction, Text Mining and NLP Ap..."
733,RUN through the Streets: A New Dataset and Bas...,4014),"Information Extraction, Text Mining and NLP Ap..."
734,Context-Aware Conversation Thread Detection in...,162),"Information Extraction, Text Mining and NLP Ap..."


In [4]:
set(df['area']) # there is no interpretability area

{'Dialog and Interactive Systems',
 'Dialog and Interactive Systems, Machine Translation and Multilinuality, Phonology, Morphology, and Word Segmentation, Speech, Vision, Robotics, Multimodal and Grounding, Tagging, Chunking, Syntax and Parsing',
 'Dialog and Interactive Systems, Speech, Vision, Robotics, Multimodal and Grounding',
 'Discourse and Pragmatics',
 'Discourse and Pragmatics, Linguistic Theories, Textual Inference, Question Answering, Summarization and Generation',
 'Discourse and Pragmatics, Summarization and Generation',
 'Discourse, Summarization, and Generation',
 'Generation',
 'Information Extraction',
 'Information Extraction, Information Retrieval and Document Analysis, Linguistic Theories',
 'Information Extraction, Text Mining and NLP Applications, Social Media and Computational Social Science, Sentiment Analysis and Argument Mining',
 'Information Retrieval and Document Analysis',
 'Information Retrieval and Document Analysis, Lexical Semantics, Sentence-level Se

In [5]:
df['interpretability'] = None
df

,title,id,area,interpretability
0,Attending to Future Tokens for Bidirectional S...,1443,Machine Learning,None
1,Attention is not not Explanation,526,Machine Learning,None
2,Practical Obstacles to Deploying Active Learning,1176,Machine Learning,None
3,Transfer Learning Between Related Tasks Using ...,1207,Machine Learning,None
4,Insertion-based Decoding with automatically In...,TACL-1732,Machine Learning,None
...,...,...,...,...
731,Spelling-Aware Construction of Macaronic Texts...,3639),"Information Extraction, Text Mining and NLP Ap...",None
732,Towards Machine Reading for Interventions from...,3718),"Information Extraction, Text Mining and NLP Ap...",None
733,RUN through the Streets: A New Dataset and Bas...,4014),"Information Extraction, Text Mining and NLP Ap...",None
734,Context-Aware Conversation Thread Detection in...,162),"Information Extraction, Text Mining and NLP Ap...",None


In [6]:
df['area'].value_counts()

area
Machine Learning                                                                                                                                                                                                   63
Dialog and Interactive Systems, Speech, Vision, Robotics, Multimodal and Grounding                                                                                                                                 47
Discourse and Pragmatics, Linguistic Theories, Textual Inference, Question Answering, Summarization and Generation                                                                                                 46
Social Media and Computational Social Science, Text Mining and NLP Applications                                                                                                                                    46
Machine Translation and Mulitilinguality, Phonology, Morphology and Word Segmentation, Tagging, Chunking, Syntax and Parsing               

In [6]:
df['doi'] = df['title'].apply(get_acl_anthology_doi)
df['source'] = 'EMNLP2019'
df

,title,id,area,interpretability,doi,source
0,Attending to Future Tokens for Bidirectional S...,1443,Machine Learning,None,10.18653/v1/D19-1001,EMNLP2019
1,Attention is not not Explanation,526,Machine Learning,None,10.18653/v1/D19-1002,EMNLP2019
2,Practical Obstacles to Deploying Active Learning,1176,Machine Learning,None,10.18653/v1/D19-1003,EMNLP2019
3,Transfer Learning Between Related Tasks Using ...,1207,Machine Learning,None,10.18653/v1/D19-1004,EMNLP2019
4,Insertion-based Decoding with automatically In...,TACL-1732,Machine Learning,None,10.1162/tacl_a_00292,EMNLP2019
...,...,...,...,...,...,...
731,Spelling-Aware Construction of Macaronic Texts...,3639),"Information Extraction, Text Mining and NLP Ap...",None,10.18653/v1/D19-1679,EMNLP2019
732,Towards Machine Reading for Interventions from...,3718),"Information Extraction, Text Mining and NLP Ap...",None,10.18653/v1/D19-1680,EMNLP2019
733,RUN through the Streets: A New Dataset and Bas...,4014),"Information Extraction, Text Mining and NLP Ap...",None,10.18653/v1/D19-1681,EMNLP2019
734,Context-Aware Conversation Thread Detection in...,162),"Information Extraction, Text Mining and NLP Ap...",None,10.18653/v1/D19-1682,EMNLP2019


# Saving the results

In [11]:
previous_df = pd.read_csv('../data/parsed_data.csv')
previous_df = previous_df.drop('Unnamed: 0', axis=1)
previous_df

,id,title,area,interpretability,doi,source
0,187,Probabilistic FastText for Multi-Sense Word Em...,Word Semantics,NaN,10.18653/v1/P18-1001,ACL2018
1,1520,A La Carte Embedding: Cheap but Effective Indu...,Word Semantics,NaN,10.18653/v1/P18-1002,ACL2018
2,707,Unsupervised Learning of Distributional Relati...,Word Semantics,NaN,10.18653/v1/P18-1003,ACL2018
3,1553,Explicit Retrofitting of Distributional Word V...,Word Semantics,NaN,10.18653/v1/P18-1004,ACL2018
4,76,Unsupervised Neural Machine Translation with W...,Machine Translation,NaN,10.18653/v1/P18-1005,ACL2018
...,...,...,...,...,...,...
9059,3639),Spelling-Aware Construction of Macaronic Texts...,"Information Extraction, Text Mining and NLP Ap...",NaN,10.18653/v1/D19-1679,EMNLP2019
9060,3718),Towards Machine Reading for Interventions from...,"Information Extraction, Text Mining and NLP Ap...",NaN,10.18653/v1/D19-1680,EMNLP2019
9061,4014),RUN through the Streets: A New Dataset and Bas...,"Information Extraction, Text Mining and NLP Ap...",NaN,10.18653/v1/D19-1681,EMNLP2019
9062,162),Context-Aware Conversation Thread Detection in...,"Information Extraction, Text Mining and NLP Ap...",NaN,10.18653/v1/D19-1682,EMNLP2019


In [14]:
# we'll delete previous rows from the conference
previous_df = previous_df[previous_df['source'] != 'EMNLP2019']
previous_df

,id,title,area,interpretability,doi,source
0,187,Probabilistic FastText for Multi-Sense Word Em...,Word Semantics,NaN,10.18653/v1/P18-1001,ACL2018
1,1520,A La Carte Embedding: Cheap but Effective Indu...,Word Semantics,NaN,10.18653/v1/P18-1002,ACL2018
2,707,Unsupervised Learning of Distributional Relati...,Word Semantics,NaN,10.18653/v1/P18-1003,ACL2018
3,1553,Explicit Retrofitting of Distributional Word V...,Word Semantics,NaN,10.18653/v1/P18-1004,ACL2018
4,76,Unsupervised Neural Machine Translation with W...,Machine Translation,NaN,10.18653/v1/P18-1005,ACL2018
...,...,...,...,...,...,...
8323,9758,Using Adversarial Attacks to Reveal the Statis...,Interpretability and Analysis of Models for NLP,True,10.18653/v1/2021.acl-short.43,ACL2021
8324,9880,Inspecting the concept knowledge graph encoded...,Interpretability and Analysis of Models for NLP,True,10.18653/v1/2021.findings-acl.263,ACL2021
8325,9436,Integrated Directional Gradients: Feature Inte...,Interpretability and Analysis of Models for NLP,True,10.18653/v1/2021.acl-long.71,ACL2021
8326,9767,Perceptual Models of Machine-Edited Text,Interpretability and Analysis of Models for NLP,True,10.18653/v1/2021.findings-acl.342,ACL2021


In [15]:
updated_df = pd.concat([previous_df, df], ignore_index=True)
updated_df

,id,title,area,interpretability,doi,source
0,187,Probabilistic FastText for Multi-Sense Word Em...,Word Semantics,NaN,10.18653/v1/P18-1001,ACL2018
1,1520,A La Carte Embedding: Cheap but Effective Indu...,Word Semantics,NaN,10.18653/v1/P18-1002,ACL2018
2,707,Unsupervised Learning of Distributional Relati...,Word Semantics,NaN,10.18653/v1/P18-1003,ACL2018
3,1553,Explicit Retrofitting of Distributional Word V...,Word Semantics,NaN,10.18653/v1/P18-1004,ACL2018
4,76,Unsupervised Neural Machine Translation with W...,Machine Translation,NaN,10.18653/v1/P18-1005,ACL2018
...,...,...,...,...,...,...
8323,3639),Spelling-Aware Construction of Macaronic Texts...,"Information Extraction, Text Mining and NLP Ap...",None,10.18653/v1/D19-1679,EMNLP2019
8324,3718),Towards Machine Reading for Interventions from...,"Information Extraction, Text Mining and NLP Ap...",None,10.18653/v1/D19-1680,EMNLP2019
8325,4014),RUN through the Streets: A New Dataset and Bas...,"Information Extraction, Text Mining and NLP Ap...",None,10.18653/v1/D19-1681,EMNLP2019
8326,162),Context-Aware Conversation Thread Detection in...,"Information Extraction, Text Mining and NLP Ap...",None,10.18653/v1/D19-1682,EMNLP2019


In [16]:
updated_df.to_csv('../data/parsed_data.csv')